<a href="https://colab.research.google.com/github/zhangling297/Substance-Use/blob/master/Copy_of_Statistical_Analysis_of_LEHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import ai
response = ai.generate_text("what is the longitudinal and latituidnal of Arizona")
print(response)

In [ ]:
# OODES Code Building Blocks, focused on tasks including merges, filters, and spatial analyses. Uing building blocks to snawer a more complicated question " HOw manty workers have a residence and a workplace within 0.5 mile of WMATA metro stations by year between 2016 and 2022?"

he Washington Metropolitan Area Transit Authority (WMATA) is the regional public transit agency that operates transportation services in the Washington, D.C. metropolitan area.

Tasks 1: read in appropriate data files, both from LODES and for geographic WMATA data
Task 2: find all census blocks with centroids within 0.5 miles of metro stations
task 3: Find al workers (by year) that have workplace and residence within 0.5 miles of metro stations



In [ ]:
import pandas as pd

# read in wac and rac data
rac_data = pd.read_csv(
    "data/ac_f5e832282bacfdea48d8690947888efb/rac_f5e832282bacfdea48d8690947888efb.csv"
)
wac_data = pd.read_csv(
    "data/ac_f5e832282bacfdea48d8690947888efb/wac_f5e832282bacfdea48d8690947888efb.csv"
)

# read in selected crosswalk
crosswalk = pd.read_csv(
    "data/ac_f5e832282bacfdea48d8690947888efb/xwalk_f5e832282bacfdea48d8690947888efb.csv"
)

# Sample 10 rows from rac_data with fixed seed for reproducibility
print(rac_data.sample(n=10, random_state=4334))

2.1. Loading Appropriate Data Files
To be able to complete the above, we will need to read in multiple files. We will need geographic data to represent the location of WMATA stations, Origin-Destination (OD) LODES files to represent workers, and LODES geographic crosswalk files to find and filter the geographic location of Census Blocks. Each of these data files are detailed below:

WMATA Station Locations: Geographic data (shapefile) on WMATA metro station locations can be downloaded from this site.

LODES OD Files: The Origin-Destination files will provide data on worker residences and workplaces.

LODES Geographic Crosswalks: The crosswalks will provide the latitude and longitude of the census blocks for spatial merging.

In [ ]:
# Reading in WMATA dta - need a package that can handle spacial data. This data includes some metro station sthat were not open during all years in the analysis. To keep consistent, remove the Silver Line Phase 2 extentsion and Potomac Yard infill station , which opened in 2023

In [ ]:
import geopandas as gpd
#read in WMATA data
metro_stations = gpd.read_file("data/wmata_metro_stations/")

#remove metro station sthat were not in service
stations_to_remove = [961, 732, 733, 734, 735, 736, 737]
metro_stations = metro_stations[~metro_stations["OBJECTID"].isin(stations_to_remove)]
#Convert to projection for contiguous USA
metro_stations = metro_stations.to_crs("ESRI:102003")
print(metro_stations.head(0))

### Data Missing: WMATA Metro Stations

The `DataSourceError` indicates that the file or directory `data/wmata_metro_stations/` does not exist, or is empty. To resolve this, you need to download the geographic data for WMATA metro stations and place it in this directory.

Please provide the URL to download the WMATA metro stations shapefile or a `.zip` archive containing the shapefile components. Once provided, I can add the necessary code to download and extract the data.

In [ ]:
# Please replace 'YOUR_DOWNLOAD_URL_HERE' with the actual URL of the WMATA metro stations data
# and adjust the extraction command if it's not a zip file.
# For example, if it's a zip file:
# !mkdir -p data
# !wget -O data/wmata_metro_stations.zip "https://www.wmata.com/"
# !unzip data/wmata_metro_stations.zip -d data/wmata_metro_stations/

print("Please provide the download URL for WMATA metro stations data to proceed.")

In [ ]:
#Reading in LODES Data
## We need LODES files for DC() Maryland, and Virginia (states that have WMATA metro stations) and across the years specified in the problem. We have chosen the job type "JT01" , which corresponds to the Primary Jobs, ( we are interested in individuals, and not jobs( for which an individual can have more than one). We will require both "main" and "auxiliary"files, since we meed workers with workplaces in the same state as their residence wwell as workers that have a residence in other states
### In addition to the OD data, wwe ned the geographic crosswalks for each of the states, we can use the functions we wrote in Loading Data to load in all the necessary LODEs data)

In [ ]:
import pandas as pd
#Read in OD data
all_od = []
states = ["dc", "va", "md"]

# Years between 2016 (include) and 2023 (exclusive)
year_range = list(range(2016, 2023))
for state in states:
  for year in year_range:
    main = read_lodes_data(
        data_type="od",
        state=state,
        segment=None,
        job_type="JT01",
        year=year, main=True,
    )
    main["year"]= yearall_od.append(aux)

# Concatenate all OD files
all_od = pd.concat(all_od)

# Read in crosswalk files
all_crosswalks = []
for state in states:
  all_crosswalks.append(read_crosswalk(state=state, cols=["blklatdd", "blklondd"]))
#Concatenate all crosswalks
all_crosswalks = pd.concat(all_crosswalks)

print(all_od.head())

In [ ]:
#

Finding Census Blocks Near Metro Stations - To find the census blocks within 0.5 miles of the metro stations,w e must adda buffer to our metro stations. Rhis code adds the buffer in meters and plots the resulting data

In [ ]:
import matplotlib.pyplot as plt
#BUffer metro stations by 805 meters (approximately 0.5 miles)
metro_stations["geometry"] = metro_stations.geometry.buffer(805)

#Create plot and plot staions
fig, ax = plt.subplots()
metro_stations.plt(ax=ax, color="Lightblue", edgecolor="black")
#Add title

ax.set_title("Buffered WMATA Metro Stations", fontsize=14)
ax.grid(True, linestyle="--", linewidth=0.5, aipha=0.7)




Next, find the Census blocks which have internal points within the buffered metro stations. We canuse a spatial join between our metro stations and teh crosswalks. First wer need to convert the projection of the crosswalk file to the same projection as the metro stations, then we we can use a spatial join to find all the blocks that have internal points within the buffer. . We do this by dinging the blocks that have internal ponts that "intersect" with the buffered WMATA polygons

In [ ]:
#Convert crosswalk to geographic dataframe
all_crosswalks,
geometry=gpd.GeoDataFrame(
    all_crosswalks,
    geometry=gpd.points_from_xy(all_crosswalks["blklondd"], all_crosswalks["blklatdd"]),
    crs="EPSG:4326",
)

# Convert to projection for contiguous USA
all_crosswalks = all_crosswalks.to_crs("ESRI:102003")

# Spatial join. using'interesects'
blocks_within_buffers = gpd.sjoin(
    all_crosswalks, metro_stations, predicate="intersects"
)

print(blocks_within_buffers.head())

# Find workers with residence and workplace near metro
# We have a list of Census blocks within the metro station buffers, we can limit our full data file to just those rows that represent workers with workplaces and residences within 0.5 miles of a metro station. Then, finally, we can group by year and display the result

In [ ]:
#Get list o fblocks witin the buffer
include_clocks = blocks_within_buffers["tabblk2020"].to_list()

#Filter for rows in which home and work are in buffered list
limited_od = all_od[
    (all_od["h_geocode"].isin(include_blocks))
    & (all_od["w_geocode"]).isin(include_clocks)
]

# Group and sum total jobs by year
jobs_by_year = limited_od{{"Sooo", "year"]].groupby("year".sum().reset_indesx()
print(jobs_by_year)

